First line if **magic** enabling matplotlib inline plots 

In [1]:
%matplotlib inline

Then we have round of inports:
* **pandas** is our main data storage module
* **glob** and **os** are used for filename manipulations

In [2]:
import pandas as pd
import glob
import os

In [3]:
filenames = [ os.path.splitext(wholeFilename)[0] for wholeFilename in 
             [ os.path.basename(wholePath) for wholePath in glob.glob("../input/2*.xlsx") ] ]

In [4]:
dataFiles = pd.DataFrame({"filename": filenames})
dataFiles["year"], dataFiles["pollutant"], dataFiles["resolution"] = dataFiles["filename"].str.split('_', 2).str

In [5]:
dataFiles["year"].value_counts()

2012    35
2011    35
2014    35
2013    34
2015    29
2010    25
2009    24
2008    24
2007    18
2006    18
2005    17
2004    17
2003    17
2002    14
2001    11
2000     6
Name: year, dtype: int64

In [ ]:
dataFiles["pollutant"].value_counts()

In [ ]:
dataFiles["resolution"].value_counts()

In [ ]:
dataFiles[dataFiles["resolution"] == "(PM2.5)_24g"]

In [ ]:
dataFiles[dataFiles["resolution"] == "(PM2.5)-24g"]

In [ ]:
dane = pd.read_excel("../input/2000_O3_1g.xlsx", skiprows=[1,2])

In [ ]:
stacje = pd.read_excel("../input/Metadane_wer20160914.xlsx")

In [ ]:
dane.head()

In [ ]:
dane = dane.rename(columns={"Kod stacji":"Godzina"})

In [ ]:
dane = dane.set_index("Godzina")

In [ ]:
#dane.hist(figsize=(11, 8))

In [ ]:
dane.columns.shape

In [ ]:
nazywStacji = set(dane.columns.values)

In [ ]:
stacje = stacje.set_index("Nr")

In [ ]:
stacje[(stacje["Stary Kod stacji"]).isin(nazywStacji) | (stacje["Kod stacji"]).isin(nazywStacji)]

In [ ]:
interesujaceStacje = stacje[(stacje["Stary Kod stacji"]).isin(nazywStacji) | (stacje["Kod stacji"]).isin(nazywStacji)]

In [ ]:
interesujaceStacje

In [ ]:
interesujaceStacje.shape

In [ ]:
interesujaceStacje[[u'WGS84 \u03bb E',u'WGS84 \u03c6 N']]

In [ ]:
wspolrzedne = interesujaceStacje[[u'WGS84 \u03bb E',u'WGS84 \u03c6 N']].values

In [ ]:
wspolrzedne[:,1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(wspolrzedne[:,0], wspolrzedne[:,1])

In [ ]:
import folium
map_osm = folium.Map(location=[52.069167, 19.480556], zoom_start=6)

In [ ]:
map_osm

In [ ]:
interesujaceStacje.index

In [ ]:
for index, row in interesujaceStacje.iterrows():
    print row['Nazwa stacji']
    folium.Marker([row[u'WGS84 \u03c6 N'], row[u'WGS84 \u03bb E']], popup=row['Nazwa stacji']).add_to(map_osm)

In [ ]:
map_osm

In [ ]:
jeden_dzien = dane[dane.index == "2000-06-12 08:00:00"]

In [ ]:
do_interpolacji = pd.melt(jeden_dzien)

In [ ]:
do_interpolacji.rename(columns={"variable":"Stary Kod stacji"}, inplace=True)

In [ ]:
final = do_interpolacji.merge(interesujaceStacje[[u'WGS84 \u03bb E',u'WGS84 \u03c6 N', "Stary Kod stacji"]])

In [ ]:
x = final[u'WGS84 \u03bb E'].values
y = final[u'WGS84 \u03c6 N'].values
z = final[u'value'].values

In [ ]:
x

In [ ]:
import numpy as np
from scipy.interpolate import griddata

In [ ]:
xi = np.linspace(x.min(),x.max(),100)
yi = np.linspace(y.min(),y.max(),200)

xi = np.append(xi,x)
xi.sort()
yi = np.append(yi,y)
yi.sort()

zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')

In [ ]:
(x,y), z

In [ ]:
zi

In [ ]:
# contour the gridded data, plotting dots at the randomly spaced data points.
CS = plt.contour(xi,yi,zi)
CS = plt.contourf(xi,yi,zi)
plt.colorbar() # draw colorbar
# plot data points.
plt.scatter(x,y,marker='o',c='b',s=5)
plt.show()

In [ ]:
from folium import plugins

In [ ]:
nxpoints = (x.max() - x.min()) / .001
nypoints = (y.max() - y.min()) / .001

In [ ]:
xi = np.linspace(x.min(),x.max(),int(nxpoints))
yi = np.linspace(y.min(),y.max(),int(nypoints))

In [ ]:
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')

In [ ]:
print(xi.shape)
print(yi.shape)
np.isnan(zi[4502,5000])

In [ ]:
from tqdm import tqdm

In [ ]:
xlist = []
ylist = []
zlist = []

In [ ]:
for xelement in tqdm(range(xi.shape[0])):
    for yelement in range(yi.shape[0]):
        if np.isnan(zi[yelement,xelement]):
            pass
        else:
            #tmpData = pd.DataFrame()
            #tmpData["x"] = xi[xelement]
            xlist.append(xi[xelement])
            ylist.append(yi[yelement])
            zlist.append(zi[yelement,xelement])
            #tmpData["y"] = yi[yelement]
            #tmpData["z"] = zi[yelement,xelement]
            #dataForHeatmap.append(tmpData, ignore_index=True)

In [ ]:
dataForHeatmap = pd.DataFrame({"x":xlist, "y":ylist, "z":zlist})

In [ ]:
dataForHeatmap

In [ ]:
#plugins.HeatMap(zip(ylist, xlist, zlist)).add_to(map_osm)

In [ ]:
#map_osm

In [ ]:
file

In [ ]:
[ basename(wholeFilename) for wholeFilename in glob.glob("../input/2*.xlsx") ]


In [ ]:
["asdfasdf", "asdfa", "asdf"]

In [ ]:
list.